In [188]:
"""My suggestion will be to try to generate sub-dataframes for each class, 
in these sub-dataframes each row has same structure and columns are all named. 
If you can do that, you then try to created some sort of mapping from column 
names to you class property names, and use that mapping to populate your data 
classes. If you can do that, most of the hard coded column numbers will be gone, 
and column order doesn't matter anymore."""

"My suggestion will be to try to generate sub-dataframes for each class, \nin these sub-dataframes each row has same structure and columns are all named. \nIf you can do that, you then try to created some sort of mapping from column \nnames to you class property names, and use that mapping to populate your data \nclasses. If you can do that, most of the hard coded column numbers will be gone, \nand column order doesn't matter anymore."

In [35]:
import pandas as pd 
import numpy as np
import os, shutil
import inspect

from tqdm import tqdm

from dataclasses import dataclass, field
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

warnings = 0
warn = 0


In [36]:
# find file dir to save files base off PMID

def get_box_dir(box_dir, pmid):
    box_base = os.path.abspath(os.path.expanduser(os.path.expandvars(box_dir)))
    depth = 3

    for dirpath, dirnames, filenames in os.walk(box_base):
        if dirpath[len(box_base):].count(os.sep) < depth:
            if "PMID_"+pmid in dirnames:
                return(os.path.join(dirpath,"PMID_"+pmid))


In [37]:
# directory Information (**kawrgs)
PMID = "34861167"


# Registry
sheet_name = 'SeroNet Registry Template'
map_sheet = 'Registry Definitions'

box_base = "~/Library/CloudStorage/Box-Box/SeroNet Public Data"

BASE_DIR = get_box_dir(box_base, PMID)
print(BASE_DIR)

file = "registry" + PMID
df_path = os.path.join(BASE_DIR,'templated_data', file + ".xlsx")

# dictionary info
registryToImmportDict_file = "./dictionary/registryToBasic.csv"

registryToImmportDict = pd.read_csv(registryToImmportDict_file,
                                    header=None, 
                                    index_col=0, 
                                    squeeze=True).to_dict()

# ImmPort Templates (link to web?)
PATH_basic_stdy_template = './template/basic_study_design.xlsx'
PATH_protocols = './template/protocols.xlsx'
PATH_experiments = './template/experiments.xlsx'


# Automate output... 
OUT_DIR = os.path.join(BASE_DIR, 'ImmPort_templates')
PATH_pmid_basic_stdy_template = f'./PMID{PMID}_study.xlsx'
EXP_TEMPLATE = f'experiments{PMID}'
PROTOCOL_TEMPLATE = f'protocol{PMID}'

# Make Dir if it does not exist
try:
    os.mkdir(OUT_DIR)
except FileExistsError:
    print('directory already exists')
    pass

# import file
book = load_workbook(df_path)
registry = book[sheet_name]
registry.delete_cols(1)


/Users/liualg/Library/CloudStorage/Box-Box/SeroNet Public Data/CA260517_BOYD/PMID_34861167


In [38]:
# add automate validation -- talk to Mani and Elaine
#   - error reporting through classes 
# no empty cell 
# date format (datetime) yyyy-mm-dd
# Error reporting 


In [39]:
# Creating a class for each part in the Excel Doc. 
# have a defult NA
# Check for PMIDXXXX
@dataclass
class study:
    """1 Column to the right"""
    Study_Identifier: str = None
    Study_Name: str = None
    Publication_Title: str = None
    Study_Objective: str = None
    Description: str = None
    Primary_Institution_Name: str = None  
    SARS_CoV_2_Vaccine_Type: str = None
    NAME : str = 'study'

@dataclass
class study_details:
    Age_Unit: str = 'Years'
    Enrollment_Start_Date: str = None #this is a datetime object
    Enrollment_End_Date: str = None #this is a datetime object
    Clinical_Outcome_Measure: str = None
    Study_Type: str = None
    Number_of_Study_Subjects: str = None
    Minimum_Age: int = None
    Maximum_Age: int = None
    Assay_Type: list = None
    ImmPortNAME : str = 'study'
    
#     def __bool__(self):
#         return bool(self.Age_Unit or self.Enrollment_Start_Date or self.Maximum_Age)
        
        

@dataclass
class study_categorization:
    """1 Column to the right"""
    Research_Focus: str = None
    ImmPortNAME : str = 'study_categorization'

@dataclass
class study_design:
    """1 Column to the right"""
    Clinical_Study_Design: str = None
    in_silico_Model_Type: str = None 
    ImmPortNAME : str = 'NA'
        
@dataclass
class condition_or_disease:
    """1 Column to the right"""
    Reported_Health_Condition: list = None
    ImmPortNAME : str = 'study_2_condition_or_disease'
        
@dataclass
class arm_or_cohort: #these can be a list, or we can do multiple ones 
    """1 row below, 4 Columns"""
    User_Defined_ID: list = None
    Name: list = None
    Description: list = None
    Type: list = None
    ImmPortNAME : str = 'arm_or_cohort'

@dataclass
class study_personnel:
    """1 row below, 11 Columns"""
    User_Defined_ID: list = None 
    Honorific: list = None
    Last_Name: list = None
    First_Name: list = None
    Suffixes: list = None
    Organization: list = None
    ORCID_ID: list = None
    Email: list = field(default_factory=list)
    Title_In_Study: list = field(default_factory=list)
    Role_In_Study: list = field(default_factory=list)
    Site_Name: list = field(default_factory=list)
    ImmPortNAME : str = 'study_personnel'

@dataclass
class planned_visit:
    """1 row below, 7 Columns"""
    User_Defined_ID: list = field(default_factory=list)
    Name: list = field(default_factory=list)
    Order_Number: list = field(default_factory=list)
    Min_Start_Day: list = field(default_factory=list)
    Max_Start_Day: list = None
    Start_Rule: list = None
    End_Rule: list = None
    ImmPortNAME : str = 'planned_visit'

@dataclass
class inclusion_exclusion:
    """1 row below, 3 Columns"""
    User_Defined_ID: list = None
    Criterion: list = field(default_factory=list)
    Criterion_Category: list = field(default_factory=list)
    ImmPortNAME : str = 'inclusion_exclusion'
    
@dataclass
class protocols:
    User_Defined_ID: str
    File_Name: str
    Name: str
    Description: str = None
    Type: str = None
    ImmPortNAME : str = 'study_2_protocol'
        
@dataclass
class study_file:
    File_Name: list = field(default_factory=list)
    Description: list = field(default_factory=list)
    Study_File_Type: list = field(default_factory=list)
    ImmPortNAME : str = 'study_file'
        
@dataclass
class study_link:
    Name: list = field(default_factory=list)
    Value: list = field(default_factory=list)
    ImmPortNAME : str = 'study_link'
        
@dataclass
class study_pubmed:
    Pubmed_ID: list
    DOI: int = None
    Title: str = None
    Journal: str = None
    Year: int = None
    Month: int = None
    Issue: str = None
    Pages: str = None
    Authors: str = None
    ImmPortNAME : str = 'study_pubmed'
        
@dataclass
class other_info:
    keyword: list = None
        


In [40]:
# Return start index for each section in the SeroNet Registry Template 
def get_sections(ws, class_names):
    temp = []
    for i in range(1,ws.max_row):
        if ws["A"][i].value in class_names:
            temp.append(i+1)
    return temp


In [41]:
def column_header_index(sheet):
    count = 0
    row_index = 1

    for row in sheet.iter_rows():
        if [cell.value for cell in row] == [None]*sheet.max_column:
            break
        row_index += 1
    
    return row_index

def row_header_index(sheet):
    count = 0
    col_index = 1

    for col in sheet.iter_cols():
        if [cell.value for cell in col] == [None]*sheet.max_row:
            break
        col_index += 1
    
    return col_index

def remove_excess(sheet):
    sheet.delete_rows(column_header_index(sheet),sheet.max_row)
    sheet.delete_cols(row_header_index(sheet),sheet.max_column)
    return sheet


In [42]:
# Class names
class_names = ['study','study_details', 'study_categorization', 'study_design','condition_or_disease',
               'arm_or_cohort','study_personnel', 'planned_visit', 'inclusion_exclusion',
               'protocol', 'study_file', 'study_link', 'study_pubmed']

In [43]:
sp = get_sections(registry, class_names)
sp

[10, 19, 32, 35, 39, 43, 50, 54, 62, 67, 71, 75, 79]

In [44]:
# Looping through teach section in the Registy template
for section_number in range(len(sp)-1):
    temp_wb = Workbook()
    temp_ws = temp_wb.active

    #making a temp workbook to store each section. This will be turned into df
    for i in registry.iter_rows(values_only = True,
                                min_row = sp[section_number]+1,
                                max_row = sp[section_number+1]-1):
        temp_ws.append(i)
        
    max_row = temp_ws.max_row
    max_col = temp_ws.max_column
    remove_excess(temp_ws)
#     print("Rows:", max_row, "->", temp_ws.max_row)
#     print("Columns:", max_col, "->", temp_ws.max_column)
    
    df = pd.DataFrame(temp_ws.values)
    sub_section = registry.cell(row=sp[section_number], column = 1).value
#     print(sub_section)

    if sub_section == 'study':
        df.index = df[0]
        df.drop(df.columns[0], axis=1, inplace=True)

        STUDY = study(
            df.loc['Study Identifier'][1],
            df.loc['Study Name'][1],
            df.loc['Publication Title'][1],
            df.loc['Study Objective'][1],
            df.loc['Description'][1],
            df.loc['Primary Institution Name'][1],
            df.loc['SARS-CoV-2 Vaccine Type '][1]
            )
        
    elif sub_section == 'study_details':
        df.index = df[0]
        df.drop(df.columns[0], axis=1, inplace=True)

        STUDY_DETAILS = study_details(
            df.loc['Age Unit'][1],
            df.loc['Enrollment Start Date'][1],
            df.loc['Enrollment End Date'][1],
            df.loc['Clinical Outcome Measure'][1],
            df.loc['Study Type'][1],
            df.loc['Number of Study Subjects'][1],
            df.loc['Minimum Age'][1],
            df.loc['Maximum Age'][1],
            list(df.loc['Assay Type'][1])
            )
        
        
    elif sub_section == 'study_categorization':
        df.index = df[0]
        df.drop(df.columns[0], axis=1, inplace=True)

        STUDY_CATEGORIZATION = study_categorization(
            df.loc['Research Focus'][1]
        )


    elif sub_section == 'study_design':
        df.index = df[0]
        df.drop(df.columns[0], axis=1, inplace=True)

        if (df.shape != (2,0)):

            STUDY_DESIGN = study_design(
                df.loc['Clinical Study Design'][1],
                df.loc['in silico Model Type'][1]
            )
        else:
            study_des = study_design()

    elif sub_section == 'condition_or_disease':
        df.index = df[0]
        df.drop(df.columns[0], axis=1, inplace=True)

        COD = condition_or_disease(
            df.loc['Reported Health Condition'][1]
        )

    elif sub_section == 'arm_or_cohort':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])
        
        AOC = arm_or_cohort(
            df['User Defined ID'],
            df['Name'],
            df['Description'],
            df['Type']
        )

    elif sub_section == 'study_personnel':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])
        
        STUDY_PERSONNEL = study_personnel(
            df['User Defined ID'],
            df['Honorific'],
            df['Last Name'],
            df['First Name'],
            df['Suffixes'],
            df['Organization'],
            df['ORCID ID'],
            df['Email'],
            df['Title In Study'],
            df['Role In Study'],
            df['Site Name']
        )

    elif sub_section == 'planned_visit':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        PLANNED_VISIT = planned_visit(
            df['User Defined ID'],
            df['Name'],
            df['Order Number'],
            df['Min Start Day'],
            df['Max Start Day'],
            df['Start Rule'],
            df['End Rule']                                
    )

    elif sub_section == 'inclusion_exclusion':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        INCLUSION_EXCLUSION = inclusion_exclusion(
            df['User Defined ID'],
            df['Criterion'],
            df['Criterion Category']  
        )


    elif sub_section == 'protocol':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        PROTOCOLS = protocols(
            df['Protocol ID'],
            df['File Name'],
            df['Name'],
            df['Description'],
            df['Type'],
        )

    elif sub_section == 'study_file':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        STUDY_FILE = study_file(
            df['File Name'],
            df['Description'],
            df['Study File Type']  
        )

    elif sub_section == 'study_link':
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        STUDY_LINK = study_link(
            df['Name'],
            df['Value']
        )

    else:
        print('sub-section does not exist')

STUDY_PUBMED = study_pubmed(
    registry.cell(row = sp[len(sp)-1]+2, column=1).value
)
    # add keyword
OTHER_INFO = other_info(
    registry.cell(row = sp[len(sp)-1]+3, column=2).value
)

### I need to add testing for correct values ###

- characters limit... but it also looks within limit
- datetime
- PMIDXXXXX_file-01
- check for pre-uploaded data


In [45]:
from dataclasses import dataclass, fields
import datetime

#https://stackoverflow.com/questions/50563546/validating-detailed-types-in-python-dataclasses

    
@dataclass
class test:
    name : str = None
    dics : int= None
    date : datetime.datetime= None
    
    
#     def __post_init__(self):
#         if self.is_good:
#             assert not self.is_bad
            
#     def __bool__(self):
#         return any(self.name, self.dict1)
        
#     def __post_init__(self) -> None:
# #         if self.name or self.dics is None:
# #             print("None Value in thing")
            
#         for f in fields(self):
#             if getattr(test, f.name) is None:
#                 print("Missing data for:", f.name)

In [46]:
me = test(name = 17.45, dics = 1)
me

for f in fields(me):
    print(getattr(me, f.name) is None)

False
False
True


# Sending back to ImmPort Template #

In [47]:
# TO DO:
"""
Intervention Agent: needs to have a value if not NA
Protocol ID: Protocol ID.txt??
arm_or_cohort: is missing LOL (elaine)
inclusion_exclusion: needs data ... (elaine)

"""


'\nIntervention Agent: needs to have a value if not NA\nProtocol ID: Protocol ID.txt??\narm_or_cohort: is missing LOL (elaine)\ninclusion_exclusion: needs data ... (elaine)\n\n'

In [48]:
ImmPortClassNames = ['study', 'study_categorization', 'study_2_condition_or_disease',
               'arm_or_cohort','study_personnel', 'planned_visit', 'inclusion_exclusion',
               'study_2_protocol', 'study_file', 'study_link', 'study_pubmed'] 

In [49]:
shutil.copy(PATH_basic_stdy_template, PATH_pmid_basic_stdy_template)
basic_stdy_template = load_workbook(PATH_pmid_basic_stdy_template)
bst_ws = basic_stdy_template['basic_study_design.txt']

se = get_sections(bst_ws, ImmPortClassNames)


/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [50]:
def add_df(ws, input_df_class, add_header = True, stagger = 0):
    
    if isinstance(input_df_class, pd.DataFrame): #checking to see if it is a class
        df = input_df_class

    else: #if its a dataframe - just overwrite df to be the dataframe
                
        if input_df_class.ImmPortNAME == 'study_pubmed':
            df = pd.DataFrame({**vars(input_df_class)}, index=[0]).drop(['ImmPortNAME'], axis = 1)
        else:
            df = pd.DataFrame({**vars(input_df_class)}).drop(['ImmPortNAME'], axis = 1)
        
    df.columns = [i.replace("_"," ") for i in df.columns]

    if add_header == True:
        ws.append([])
        ws.append([input_df_class.ImmPortNAME])
        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(['']*stagger + r)
    else:
        for r in dataframe_to_rows(df, index=False, header=False):
            ws.append(['']*stagger + r)
        

In [51]:
isinstance(STUDY_PERSONNEL, pd.DataFrame)

False

## Basic Study Template ##

In [52]:
temp_wb = Workbook()
temp_ws = temp_wb.active

#making a temp workbook to store each section. This will be turned into df

# This is the first part of the df
for i in bst_ws.iter_rows(values_only = True,
                          max_row = se[2]+1):
    temp_ws.append(i)
    
# Starting at Arm or Cohort  
add_df(temp_ws, STUDY_PERSONNEL)
add_df(temp_ws, PLANNED_VISIT)
add_df(temp_ws, INCLUSION_EXCLUSION)

# Add protocol
temp_ws.append([])
temp_ws.append([PROTOCOLS.ImmPortNAME])
temp_ws.append(['Protocol ID', PROTOCOLS.Name[1]])

add_df(temp_ws, STUDY_FILE)
add_df(temp_ws, STUDY_LINK)

add_df(temp_ws, STUDY_PUBMED)

# pd.DataFrame(temp_ws.values).tail()

In [53]:
PROTOCOLS.Name[1]

'PMID34861167_protocol-01'

In [54]:
registryToImmportDict = pd.read_csv(registryToImmportDict_file, header=None, index_col=0, squeeze=True).to_dict()
registryDict = {**vars(STUDY), **vars(STUDY_DETAILS), **vars(STUDY_CATEGORIZATION), **vars(STUDY_DETAILS), **vars(COD)}

# Looping through ImmPort Template to get the correct order of the 'study' section
for se_number in range(se[0],se[3]):    
#     print(type(temp_ws["A"][se_number].value))
    if temp_ws["A"][se_number].value != None and registryToImmportDict.get(temp_ws["A"][se_number].value) != None:
        
        # Using a mapping key + using info in our classes to map the data
        reg_key = registryToImmportDict.get(temp_ws["A"][se_number].value).replace(' ',"_")
        
        try:
            # If input is a list, we will turn it into a string (since it can be a list)
            if type(registryDict.get(reg_key)) == list:
                temp_ws["B"][se_number].value = ', '.join(registryDict.get(reg_key))
            else:
                temp_ws["B"][se_number].value = registryDict.get(reg_key)

        except:
            print(f"{reg_key} did not work")
            
#     else:
#         print('Row')


In [55]:
bsd = pd.DataFrame(temp_ws.values).replace({None: '', 'None': ''})
bsd.head()

,0,1,2,3,4,5,6,7,8,9,10
0,basic_study_design,Schema Version 3.34,,,,,,,,,
1,Please do not delete or edit this column,The basic study design template defines and an...,,,,,,,,,
2,Column Name,,,,,,,,,,
3,,,,,,,,,,,
4,study,,,,,,,,,,


In [56]:
bsd.to_excel(os.path.join(OUT_DIR, f'PMID{PMID}_study.xlsx'), index=False, header = False)
bsd.to_csv(os.path.join(OUT_DIR, f'PMID{PMID}_study.txt'), index=False, header = False, sep ='\t')

In [57]:
# acd = {**vars(study_main), **vars(study_details)}
# acd.get('Primary_Institution_Name')

### TO DO ###
Experiments: DO<br>
~Comes from Assay section~<br>
~in 3rd tab under registary deffinitions: map the assay type to the description (when you port it over)~<br>
~Name is the assay type (what will this be)~<br>
~User defined ID also auto generated~<br>
~This template will be generated from "assay type" broken by delimitars~ <br>

Subjects: NO
- comes from AOC section 

~Protocol: DO~

~Basic study design: DO~

## Protocol ##

In [69]:
protocol_ws = load_workbook(PATH_protocols)['protocols.txt']
protocol_ws = remove_excess(protocol_ws)
add_df(protocol_ws, PROTOCOLS, add_header = False, stagger = 1)

In [70]:
protocol_df = pd.DataFrame(protocol_ws.values).replace({None: '', 'None': ''})
protocol_df.to_csv(os.path.join(OUT_DIR,f'{PROTOCOL_TEMPLATE}.txt'),
                   header = False, 
                   index = False,
                   sep = '\t')
try:
    shutil.copy(os.path.join(BASE_DIR,'templated_data', f'{PROTOCOLS.Name[1]}.txt'),
                os.path.join(OUT_DIR,f'{PROTOCOLS.Name[1]}.txt'))
except:
    print(f'File: {PROTOCOLS.Name[1]}.txt does not exist')

File: PMID34861167_protocol-01.txt does not exist


In [64]:
protocol_df

,0,1,2,3,4,5
0,protocols,Schema Version 3.33,,,,
1,Please do not delete or edit this column,,,,,
2,Column Name,User Defined ID,File Name,Name,Description,Type
3,,PMID34861167_protocol-01,PMID34861167_protocol-01.txt,PMID34861167_protocol-01,,Not Specified


## Experiment ##

In [65]:
# creating a map of the assay types to the SeroNet descriptors 
reg_description = pd.read_excel(df_path, sheet_name = map_sheet)
descriptions = dict(zip(reg_description['Unnamed: 1'][4:], reg_description['Unnamed: 2'][4:]))

#creating a df to add into the worksheet
experiments_df = pd.DataFrame({
    'Column Name': ['']*len(STUDY_DETAILS.Assay_Type),
    'User Defined ID': [f'PMID{PMID}_exp-0'+str(i+1) for i,k in enumerate(STUDY_DETAILS.Assay_Type)],
    'Name': [STUDY_DETAILS.Assay_Type[i] for i, k in enumerate(STUDY_DETAILS.Assay_Type)],
    'Description': [descriptions.get(k) for i, k in enumerate(STUDY_DETAILS.Assay_Type)],
    'Measurement Technique': [STUDY_DETAILS.Assay_Type[i] for i, k in enumerate(STUDY_DETAILS.Assay_Type)],
    'Study ID': [STUDY.Study_Identifier]*len(STUDY_DETAILS.Assay_Type),
    'Protocol ID(s)': [PROTOCOLS.User_Defined_ID[1]]*len(STUDY_DETAILS.Assay_Type)
})

In [66]:
# loading experiment template and removing excess rows and columns
experiment_ws = load_workbook(PATH_experiments)['experiments.txt']
experiment_ws = remove_excess(experiment_ws)

# adding df to bottom of ws
add_df(experiment_ws, experiments_df, add_header = False)


/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [67]:
experiments_df = pd.DataFrame(experiment_ws.values).replace({None: '', 'None': ''})
experiments_df.to_csv(os.path.join(OUT_DIR,f'{EXP_TEMPLATE}.txt'),
                      header = False, 
                      index = False,
                      sep = '\t')

In [68]:
experiments_df

,0,1,2,3,4,5,6
0,experiments,Schema Version 3.34,,,,,
1,Please do not delete or edit this column,,,,,,
2,Column Name,User Defined ID,Name,Description,Measurement Technique,Study ID,Protocol ID(s)
3,,PMID34861167_exp-01,Chemiluminescent Assay,Study has reported results from one or more Ch...,Chemiluminescent Assay,PMID34861167_study,PMID34861167_protocol-01
4,,PMID34861167_exp-02,Virus Neutralization Assay,Study has reported results from one or more Vi...,Virus Neutralization Assay,PMID34861167_study,PMID34861167_protocol-01
5,,PMID34861167_exp-03,IgH Sequencing,,IgH Sequencing,PMID34861167_study,PMID34861167_protocol-01


### ERRORS ###
- experiments34188786.txt	6	Error	Value reported in template does not match preferred vocabulary. See template documentation for preferred vocabulary terms.	The Lookup Table check "lk_exp_measurement_tech" in field "Measurement Technique" "Virus Neutralization Assay".  The row has ID field(s) "User Defined ID" and value "PMID34188786_exp-02".
